In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc

# Patch for working directory
path = Path.cwd()

# Get the files from the path provided in the OP
files = Path(path).glob('*.csv')  # .rglob to get subdirectories

In [2]:
#Find all csv files in directory and concatenate them
dfs = list()
names = list()
for f in files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    names.append(f.stem)
    dfs.append(data)
    

df_raw = pd.concat(dfs, ignore_index=True)

In [3]:
#drop last column taht shows csv filename source:
del df_raw[df_raw.columns[-1]]

#df_raw

In [4]:
print(df_raw.shape)

df_raw['marker'].value_counts()

(78377, 129)


Attack      55663
Natural     18309
NoEvents     4405
Name: marker, dtype: int64

In [5]:
#----> UNBALANCED DATA

In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78377 entries, 0 to 78376
Columns: 129 entries, R1-PA1:VH to marker
dtypes: float64(113), int64(15), object(1)
memory usage: 77.1+ MB


In [7]:
# Look at datatypes of coloumns.
def show_dtypes(df):
    for index in range(len(df.dtypes)):
        print(f'{df.columns[index]} -> {df.dtypes[index]}' )
        
show_dtypes(df_raw)

R1-PA1:VH -> float64
R1-PM1:V -> float64
R1-PA2:VH -> float64
R1-PM2:V -> float64
R1-PA3:VH -> float64
R1-PM3:V -> float64
R1-PA4:IH -> float64
R1-PM4:I -> float64
R1-PA5:IH -> float64
R1-PM5:I -> float64
R1-PA6:IH -> float64
R1-PM6:I -> float64
R1-PA7:VH -> float64
R1-PM7:V -> float64
R1-PA8:VH -> float64
R1-PM8:V -> float64
R1-PA9:VH -> float64
R1-PM9:V -> float64
R1-PA10:IH -> float64
R1-PM10:I -> float64
R1-PA11:IH -> float64
R1-PM11:I -> float64
R1-PA12:IH -> float64
R1-PM12:I -> float64
R1:F -> float64
R1:DF -> float64
R1-PA:Z -> float64
R1-PA:ZH -> float64
R1:S -> int64
R2-PA1:VH -> float64
R2-PM1:V -> float64
R2-PA2:VH -> float64
R2-PM2:V -> float64
R2-PA3:VH -> float64
R2-PM3:V -> float64
R2-PA4:IH -> float64
R2-PM4:I -> float64
R2-PA5:IH -> float64
R2-PM5:I -> float64
R2-PA6:IH -> float64
R2-PM6:I -> float64
R2-PA7:VH -> float64
R2-PM7:V -> float64
R2-PA8:VH -> float64
R2-PM8:V -> float64
R2-PA9:VH -> float64
R2-PM9:V -> float64
R2-PA10:IH -> float64
R2-PM10:I -> float64
R2-P

In [8]:
# .copy() is very important otherwise its just a pointer
df = df_raw.copy()

In [9]:
def marker_to_numeric(x):
        if x=='NoEvents': return 0
        if x=='Natural':   return 1
        if x=='Attack': return 2

df['marker'] = df['marker'].apply(marker_to_numeric)
#df

In [10]:
np.where(df.values >= np.finfo(np.float64).max)

(array([  738,   739,   753, ..., 78352, 78353, 78353], dtype=int64),
 array([26, 26, 26, ..., 55, 26, 55], dtype=int64))

In [11]:
np.isnan(df.values.any())

False

In [12]:
# Replace infinite updated data with nan
df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
df.dropna(inplace=True)

print(df.shape)

df['marker'].value_counts()

(72073, 129)


2    51445
1    16223
0     4405
Name: marker, dtype: int64

In [13]:
# Due to cleaning up NaNs and infinites we lost about 6000 datapoints, which is not very harmful

In [14]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

#This only copies a Series Object, no pandas Dataframe!
df_marker = df["marker"].copy()
#Converting to Pandas Dataframe
df_marker = pd.DataFrame(df_marker)

#Delete Marker column because "marker" should not be normalized
del df[df.columns[-1]]

#Initialize SCaler
scaler = StandardScaler()
#Scale data
data_normalized = scaler.fit_transform(df)
#insert normalized data in former pandas frame
df_normalized_nomark = pd.DataFrame(data_normalized, columns=df.columns)

#concat "marker" column with normalized rest dataframe, index of marker frame is overwritten
df_normalized = pd.concat([df_normalized_nomark, df_marker.set_index(df_normalized_nomark.index)], axis=1)

#print(df_normalized_nomark)

#print(isinstance(df, pd.DataFrame))

#print(isinstance(df_marker, pd.DataFrame))

#print(df_normalized)


In [15]:
"""
# Replace infinite updated data with nan
df_normalized.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
df_normalized.dropna(inplace=True)

print(df_normalized.shape)

df['marker'].value_counts()
"""

"\n# Replace infinite updated data with nan\ndf_normalized.replace([np.inf, -np.inf], np.nan, inplace=True)\n# Drop rows with NaN\ndf_normalized.dropna(inplace=True)\n\nprint(df_normalized.shape)\n\ndf['marker'].value_counts()\n"

In [16]:

y = df_normalized['marker']

X = df_normalized.drop(['marker'],axis =1)


In [17]:

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape


((43243, 128), (43243,), (14415, 128), (14415,), (14415, 128), (14415,))

In [30]:
print(type(y_train))

<class 'pandas.core.series.Series'>


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [31]:
def XnumpyToTensor(x_data_df):
    x_data_np = np.array(x_data_df.values, dtype=np.float32)
    print(x_data_np.shape)
    print(type(x_data_np))
    
    if use_cuda:
        lgr.info ("Using the GPU")    
        X_tensor = Variable(torch.from_numpy(x_data_np).cuda()) # Note the conversion for pytorch    
    else:
        lgr.info ("Using the CPU")
        X_tensor = Variable(torch.from_numpy(x_data_np)) # Note the conversion for pytorch
        
    print(type(X_tensor.data)) # should be 'torch.cuda.FloatTensor'            
    print((X_tensor.data.shape)) # torch.Size([108405, 29])
    return X_tensor

# Convert the np arrays into the correct dimention and type
# Note that BCEloss requires Float in X as well as in y
def YnumpyToTensor(y_data_df): 
    y_data_np = np.array(y_data_df.values, dtype=np.float32)
    y_data_np=y_data_np.reshape((y_data_np.shape[0],1)) # Must be reshaped for PyTorch!
    print(y_data_np.shape)
    print(type(y_data_np))

    if use_cuda:
        lgr.info ("Using the GPU")            
    #     Y = Variable(torch.from_numpy(y_data_np).type(torch.LongTensor).cuda())
        Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor).cuda()  # BCEloss requires Float        
    else:
        lgr.info ("Using the CPU")        
    #     Y = Variable(torch.squeeze (torch.from_numpy(y_data_np).type(torch.LongTensor)))  #         
        Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor)  # BCEloss requires Float        

    print(type(Y_tensor.data)) # should be 'torch.cuda.FloatTensor'
    print(y_data_np.shape)
    print(type(y_data_np))    
    return Y_tensor

In [27]:
# References:
# https://github.com/vinhkhuc/PyTorch-Mini-Tutorials/blob/master/5_convolutional_net.py
# https://gist.github.com/spro/c87cc706625b8a54e604fb1024106556

import logging

lgr = logging.getLogger(__name__)

use_cuda=False
X_tensor_train= XnumpyToTensor(X_train) # default order is NBC for a 3d tensor, but we have a 2d tensor
X_shape=X_tensor_train.data.size()

# Dimensions
# Number of features for the input layer
N_FEATURES=X_train.shape[1]
# Number of rows
NUM_ROWS_TRAINING=X_train.shape[0]
# this number has no meaning except for being divisable by 2
N_MULT_FACTOR=8 # min should be 4
# Size of first linear layer
N_HIDDEN=N_FEATURES * N_MULT_FACTOR
# CNN kernel size
N_CNN_KERNEL=3
MAX_POOL_KERNEL=4

DEBUG_ON=False

def debug(x):
    if DEBUG_ON:
        print ('(x.size():' + str (x.size()))
    
class Net2(nn.Module):    
    def __init__(self, n_feature, n_hidden, n_output, n_cnn_kernel, n_mult_factor=N_MULT_FACTOR):
        super(Net2, self).__init__()
        self.n_feature=n_feature
        self.n_hidden=n_hidden
        self.n_output= n_output 
        self.n_cnn_kernel=n_cnn_kernel
        self.n_mult_factor=n_mult_factor
        self.n_l2_hidden=self.n_hidden * (self.n_mult_factor - self.n_cnn_kernel + 3)
#         self.n_out_hidden=int (self.n_l2_hidden/2)
                        
        self.l1 = nn.Sequential(
            torch.nn.Linear(self.n_feature, self.n_hidden),
            torch.nn.Dropout(p=1 -.85),            
            torch.nn.LeakyReLU (0.1),            
            torch.nn.BatchNorm1d(self.n_hidden, eps=1e-05, momentum=0.1, affine=True)            
        )                
        self.c1= nn.Sequential(            
            torch.nn.Conv1d(self.n_feature, self.n_hidden, 
                            kernel_size=(self.n_cnn_kernel,), stride=(1,), padding=(1,)),
            torch.nn.Dropout(p=1 -.75),            
            torch.nn.LeakyReLU (0.1),
            torch.nn.BatchNorm1d(self.n_hidden, eps=1e-05, momentum=0.1, affine=True)        
        )                        
        self.out = nn.Sequential(
            torch.nn.Linear(self.n_l2_hidden,
                            self.n_output),  
        )                
        self.sig=nn.Sigmoid()

        
    def forward(self, x):
        debug(x)
        varSize=x.data.shape[0] # must be calculated here in forward() since its is a dynamic size        
        x=self.l1(x)                
        debug(x)
        # for CNN        
        x = x.view(varSize,self.n_feature,self.n_mult_factor)
        debug(x)
        x=self.c1(x)
        debug(x)
        # for Linear layer
        x = x.view(varSize, self.n_hidden * (self.n_mult_factor -self.n_cnn_kernel + 3))
        debug(x)
#         x=self.l2(x)                    
        x=self.out(x)   
        debug(x)
        x=self.sig(x)
        return x
    
net = Net2(n_feature=N_FEATURES, n_hidden=N_HIDDEN, n_output=3, n_cnn_kernel=N_CNN_KERNEL)   # define the network    
if use_cuda:
    net=net.cuda() # very important !!!
lgr.info(net)
b = net(X_tensor_train)
print ('(b.size():' + str (b.size())) 

(43243, 128)
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
torch.Size([43243, 128])
(b.size():torch.Size([43243, 3])


In [28]:
LR = 0.005
MOMENTUM = 0.9

optimizer = torch.optim.Adam(net.parameters(), lr=LR,weight_decay=5e-4) #  L2 regularization
loss_func=torch.nn.BCELoss() # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
if use_cuda:
    lgr.info ("Using the GPU")    
    net.cuda()
    loss_func.cuda()

lgr.info (optimizer)
lgr.info (loss_func)

In [29]:
import time
start_time = time.time()    
epochs=100
all_losses = []

X_tensor_train= XnumpyToTensor(X_train)
Y_tensor_train= YnumpyToTensor(y_train)

print(type(X_tensor_train.data), type(Y_tensor_train.data)) # should be 'torch.cuda.FloatTensor'

# From here onwards, we must only use PyTorch Tensors
for step in range(epochs):
    out = net(X_tensor_train)                 # input x and predict based on x
    cost = loss_func(out, Y_tensor_train)     # must be (1. nn output, 2. target), the target label is NOT one-hotted

    optimizer.zero_grad()   # clear gradients for next train
    cost.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
                           
    if step % 10 == 0:        
        loss = cost.data
        all_losses.append(loss)
        print(step, cost.data.cpu().numpy())        
        prediction = (net(X_tensor_train).data).float() # probabilities             
        pred_y = prediction.cpu().numpy().squeeze()
        target_y = Y_tensor_train.cpu().data.numpy()                        
        tu = ((pred_y == target_y).mean(),log_loss(target_y, pred_y),roc_auc_score(target_y,pred_y ))
        print ('ACC={}, LOG_LOSS={}, ROC_AUC={} '.format(*tu))        
                
end_time = time.time()
print ('{} {:6.3f} seconds'.format('GPU:', end_time-start_time))

%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(all_losses)
plt.show()

false_positive_rate, true_positive_rate, thresholds = roc_curve(target_y,pred_y)
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('LOG_LOSS=' + str(log_loss(target_y, pred_y)))
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.6f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([-0.1, 1.2])
plt.ylim([-0.1, 1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

(43243, 128)
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
torch.Size([43243, 128])
(43243, 1)
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
(43243, 1)
<class 'numpy.ndarray'>
<class 'torch.Tensor'> <class 'torch.Tensor'>


ValueError: Using a target size (torch.Size([43243, 1])) that is different to the input size (torch.Size([43243, 3])) is deprecated. Please ensure they have the same size.

In [ ]:
"""
class CNN1_Net(nn.Module):
    def __init__(self, kernel_size_1=5,
                stide_size=1,
                num_channels=1,
                depth_1=1,
                depth_2=5,
                kernel_size_2=5,
                num_hidden=20,
                num_labels=5
                ):
        super(CNN1_net, self).__init__()
        
        self.classifier = nn.Sequential(
        nn.Conv1d(num_channels, depth_1, kernel_size=kernel_size_1),
        #nn.ReLU(),
        #nn.MaxPool1d(1),
        #nn.Dropout(0.1),
        #nn.Conv1d(in_channels=1, out_channel=1, kernel_size=5)
        )
        self.fc1 = nn.Sequential(
        nn.Linear(depth_2*kernel_size_2, num_hidden),
        nn.ReLU(),
        nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
        nn.Linear(num_hidden, num_labels),
        nn.ReLU(),
        nn.Dropout(0.5)
        )
        
    def forward(self,x):
        x=self.classifier(x)
        #x=self.fc1(x)
        #x=self.fc2(x)
        print(x)
        return(x)
"""        


In [ ]:
class MyDataset():
    def __init__(self):
        x = df_normalized.drop(['marker'],axis =1).values
        y = df_normalized['marker'].values
        
        self.x_train = torch.tensor(x, dtype=torch.float64)
        self.y_train = torch.tensor(y, dtype=torch.float64)
        
    def __len__(self):
        return len(self.y_train)
    
    def __getitem__(self,idx):
        return self.x_train[idx], self.y_train[idx]
        
        

In [ ]:
myDs = MyDataset()
train_loader = DataLoader(myDs,batch_size=10, shuffle=False)

In [ ]:
for i, (data, labels) in enumerate(train_loader):
    print(data.shape , labels.shape)
    print(data, labels)
    break;

In [ ]:
df_normalized.head()